# Software Requirements Classification
## Decision Tree Classifier

In [31]:
import pandas as pd

df = pd.read_csv('software_requirements_extended.csv')
df['Type'] = df['Type'].apply(lambda x: 1 if x == 'FR' or x == 'F' else 0)

In [32]:
import re
import nltk  #natural language processing

nltk.download("stopwords")
from nltk.corpus import stopwords
from nltk.stem.porter import *


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/nickbres/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [33]:
pattern  = re.compile(r"[^a-zA-Z0-9]")
def req_to_words(req):
    ''' Convert tweet text into a sequence of words '''
    # convert to lower case
    text = req.lower()
    # remove non letters
    text = re.sub(pattern, " ", text)
    # tokenize
    words = text.split()
    # remove stopwords
    words = [w for w in words if w not in stopwords.words("english")]
    # apply stemming
    words = [PorterStemmer().stem(w) for w in words]
    # return list
    return words

### Model

In [34]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report

In [35]:
X = df['Requirement']
y = df['Type']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

In [36]:
vectorizer = TfidfVectorizer(analyzer=req_to_words)
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [37]:
model = DecisionTreeClassifier()
model.fit(X_train, y_train)

DecisionTreeClassifier()

In [38]:
y_pred = model.predict(X_test)
print(f'Accuracy: {accuracy_score(y_test,y_pred) * 100:.2f}%')
print(classification_report(y_test, y_pred))

Accuracy: 77.24%
              precision    recall  f1-score   support

           0       0.76      0.72      0.74       176
           1       0.78      0.81      0.80       215

    accuracy                           0.77       391
   macro avg       0.77      0.77      0.77       391
weighted avg       0.77      0.77      0.77       391


## With AdaBoost

In [39]:
from sklearn.ensemble import AdaBoostClassifier

In [40]:
ada_model = AdaBoostClassifier(
    estimator=DecisionTreeClassifier(max_depth=1),
    n_estimators=50,
    learning_rate=1
)
ada_model.fit(X_train, y_train)

AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=1),
                   learning_rate=1)

In [41]:
y_pred = ada_model.predict(X_test)
print(f'Accuracy: {accuracy_score(y_test,y_pred) * 100:.2f}%')
print(classification_report(y_test, y_pred))

Accuracy: 79.28%
              precision    recall  f1-score   support

           0       0.77      0.78      0.77       176
           1       0.82      0.80      0.81       215

    accuracy                           0.79       391
   macro avg       0.79      0.79      0.79       391
weighted avg       0.79      0.79      0.79       391
